Basert på http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [1]:
import torch
import torchvision
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class lenet_all(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=192, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dense1 = nn.Linear(in_features=192*8*8, out_features=1000)
        self.dense2 = nn.Linear(in_features=1000, out_features=10)
        
    def forward(self, x):
        
        x = F.dropout(self.conv1(x), p=0.5)
        print(f'After first conv: {x.size()}')
        x = self.pool1(x)
        print(f'After first conv+pool: {x.size()}')
        x = F.dropout(self.conv2(x), p=0.5)
        print(f'After second conv: {x.size()}')
        x = self.pool2(x)
        print(f'After second conv+pool: {x.size()}')
        x = x.view(x.size(0), -1)
        print(f'After flattening: {x.size()}')
        x = F.dropout(F.relu(self.dense1(x)), p=0.5)
        x = self.dense2(x)
        
        return F.log_softmax(x, dim=-1)

La til print statements for å sjekke størrelser på outputs.

In [4]:
net = lenet_all()

In [5]:
net.modules

<bound method Module.modules of lenet_all(
  (conv1): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv2): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (dense1): Linear(in_features=12288, out_features=1000, bias=True)
  (dense2): Linear(in_features=1000, out_features=10, bias=True)
)>

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
i, data = next(enumerate(trainloader, 0))

In [8]:
inputs, labels = data
inputs, labels = Variable(inputs), Variable(labels)
optimizer.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

After first conv: torch.Size([4, 192, 32, 32])
After first conv+pool: torch.Size([4, 192, 16, 16])
After second conv: torch.Size([4, 192, 16, 16])
After second conv+pool: torch.Size([4, 192, 8, 8])
After flattening: torch.Size([4, 12288])


**NB: Fjern prints fra `forward` før du kjører neste:**

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

## Annet eksempel

Fra tutorial: http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolution-neural-network

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=None, padding=0)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        print(f'Before first conv: {x.size()}')
        x = F.relu(self.conv1(x))
        print(f'After first conv: {x.size()}')
        x = self.pool(x)
        print(f'After first conv+pool: {x.size()}')
        x = F.relu(self.conv2(x))
        print(f'After second conv: {x.size()}')
        x = self.pool(x)
        print(f'After second conv+pool: {x.size()}')
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net2 = Net()

In [10]:
net2.modules

<bound method Module.modules of Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [11]:
net.modules

<bound method Module.modules of lenet_all(
  (conv1): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv2): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (dense1): Linear(in_features=12288, out_features=1000, bias=True)
  (dense2): Linear(in_features=1000, out_features=10, bias=True)
)>

In [12]:
inputs, labels = data
inputs, labels = Variable(inputs), Variable(labels)
optimizer.zero_grad()
outputs = net2(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

Before first conv: torch.Size([4, 3, 32, 32])
After first conv: torch.Size([4, 6, 28, 28])
After first conv+pool: torch.Size([4, 6, 14, 14])
After second conv: torch.Size([4, 16, 10, 10])
After second conv+pool: torch.Size([4, 16, 5, 5])


Sammenlign med lenet:

In [13]:
inputs, labels = data
inputs, labels = Variable(inputs), Variable(labels)
optimizer.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

After first conv: torch.Size([4, 192, 32, 32])
After first conv+pool: torch.Size([4, 192, 16, 16])
After second conv: torch.Size([4, 192, 16, 16])
After second conv+pool: torch.Size([4, 192, 8, 8])
After flattening: torch.Size([4, 12288])


Merk at conv i net2 minker størrelsen med 4. Dette pga ingen padding. 